In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys

import argparse
import numpy as np
from PIL import Image

from models.image_text_transformation import ImageTextTransformation
from utils.util import read_image_width_height, resize_long_edge

In [3]:

parser = argparse.ArgumentParser()
# parser.add_argument('--image_src', default='examples/1.jpg')
# parser.add_argument('--out_image_name', default='output/1_result.jpg')
parser.add_argument(
    '--gpt_version',
    choices=['gpt-3.5-turbo', 'gpt4'],
    default='gpt-3.5-turbo',
)
parser.add_argument(
    '--image_caption',
    action='store_true',
    dest='image_caption',
    default=True,
    help='Set this flag to True if you want to use BLIP2 Image Caption',
)
parser.add_argument(
    '--dense_caption',
    action='store_true',
    dest='dense_caption',
    default=True,
    help='Set this flag to True if you want to use Dense Caption',
)
parser.add_argument(
    '--semantic_segment',
    action='store_true',
    dest='semantic_segment',
    default=True,
    help='Set this flag to True if you want to use semantic segmentation',
)

parser.add_argument(
    '--sam_arch',
    choices=['vit_b', 'vit_l', 'vit_h'],
    dest='sam_arch',
    default='vit_b',
    help='vit_b is the default model (fast but not accurate), vit_l and vit_h are larger models',
)
parser.add_argument(
    '--captioner_base_model',
    choices=['blip', 'blip2'],
    dest='captioner_base_model',
    default='blip',
    help='blip2 requires 15G GPU memory, blip requires 6G GPU memory',
)
parser.add_argument(
    '--region_classify_model',
    choices=['ssa', 'edit_anything'],
    dest='region_classify_model',
    default='ssa',
    help='Select the region classification model: edit anything is ten times faster than ssa, but less accurate.',
)

backends = ['cuda', 'cuda:0', 'cuda:1', 'cpu']
parser.add_argument(
    '--image_caption_device',
    choices=backends,
    default='cuda:0',
    help=f'Select the device: {backends}, gpu memory larger than 14G is recommended',
)
parser.add_argument(
    '--dense_caption_device', choices=backends,
    default='cuda:0',
    help=f'Select the device: {backends}, < 6G GPU is not recommended>',
)
parser.add_argument(
    '--semantic_segment_device',
    choices=backends,
    default='cuda:1',
    help=f'Select the device: {backends}, gpu memory larger than 14G is recommended. '
         ' Make sure this model and image_caption model on same device. (Only relevant for EditAnything?)',
)
parser.add_argument(
    '--contolnet_device',
    choices=backends,
    default='cuda:0',
    help=f'Select the device: {backends}, <6G GPU is not recommended>',
)

args = parser.parse_args(args=[])


In [4]:
processor = ImageTextTransformation(args)

{'gpt_version': 'gpt-3.5-turbo',
 'image_caption': True,
 'dense_caption': True,
 'semantic_segment': True,
 'sam_arch': 'vit_b',
 'captioner_base_model': 'blip',
 'region_classify_model': 'ssa',
 'image_caption_device': 'cuda:0',
 'dense_caption_device': 'cuda:0',
 'semantic_segment_device': 'cuda:1',
 'contolnet_device': 'cuda:0'}
-------------------Welcome to the Image2Paragraph toolbox...--------------------
-----------------------------Initializing models...-----------------------------
---------------------This is time-consuming, please wait...---------------------
-------------------0.00 Initializing ImageCaptioning blip...--------------------
blip initialized on cuda:0
---------------------19.74 Initializing DenseCaptioning...----------------------
DenseCaptioning initialize is empty and device: cuda:0 is used in a weird way
----------------19.74 Initializing ImageToText gpt-3.5-turbo...-----------------
-----------------19.74 Initializing TextToImage (controlnet)...-----------

You have disabled the safety checker for <class 'diffusers.pipelines.stable_diffusion.pipeline_stable_diffusion_controlnet.StableDiffusionControlNetPipeline'> by passing `safety_checker=None`. Ensure that you abide to the conditions of the Stable Diffusion license and do not expose unfiltered results in services or applications open to the public. Both the diffusers team and Hugging Face strongly recommend to keep the safety filter enabled in all public facing circumstances, disabling it only for use-cases that involve analyzing network behavior or auditing its results. For more information, please have a look at https://github.com/huggingface/diffusers/pull/254 .


SD-ControlNet initialized on cpu with cuda:0 requested
SD-ControlNet pipeline initialized on cpu with cuda:0 requested
--------------------25.86 Initializing RegionSemantic ssa...--------------------


/home/dmcconachie/miniconda3/envs/mllm_testing/lib/python3.8/site-packages/transformers/models/oneformer/image_processing_oneformer.py:417: FutureWarning: The `reduce_labels` argument is deprecated and will be removed in v4.27. Please use `do_reduce_labels` instead.
  warnings.warn(


SemanticSegment initialized on cuda:1
----------------------55.71 Model initialization finished!----------------------


In [5]:
# import torch
# from transformers import (CLIPProcessor, CLIPModel, AutoProcessor, CLIPSegForImageSegmentation,
#                           OneFormerProcessor, OneFormerForUniversalSegmentation,
#                           BlipProcessor, BlipForConditionalGeneration)
                          
# model_name = "shi-labs/oneformer_ade20k_swin_tiny"
# oneformer_ade20k_processor = OneFormerProcessor.from_pretrained(model_name, dtype=torch.float16)
# oneformer_ade20k_model = OneFormerForUniversalSegmentation.from_pretrained(model_name)
# oneformer_ade20k_model = oneformer_ade20k_model.to(device="cuda:1", dtype=torch.float16)

In [6]:
root_path = "/home/dmcconachie/single_test_log_20230508-105542.camera.images"
filename = "00000000f1182120-000000.png"
image_path = f"{root_path}/{filename}"

# ref_image = resize_long_edge(Image.open(image_path), 384)
# width, height = read_image_width_height(img_src)

image_caption = processor.image_caption_model.image_caption(image_path)

/home/dmcconachie/miniconda3/envs/mllm_testing/lib/python3.8/site-packages/transformers/generation/utils.py:1313: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


****************************************************************************************************

Step1, blip caption:
a table with a bunch of oranges on it
****************************************************************************************************


In [7]:
dense_caption = processor.dense_caption_model.image_dense_caption(image_path)

/home/dmcconachie/miniconda3/envs/mllm_testing/lib/python3.8/site-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  /pytorch/aten/src/ATen/native/BinaryOps.cpp:467.)
  return torch.floor_divide(self, other)
/home/dmcconachie/miniconda3/envs/mllm_testing/lib/python3.8/site-packages/torch/nn/functional.py:3609: UserWarning: Default upsampling behavior when mode=linear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  warnings.warn(
/home/dmcconachie/miniconda3/envs/mllm_testing/lib/python3.8/site-packages/to

****************************************************************************************************
Step2, Dense Caption:

wooden cutting board: [1, 3, 380, 211]; four oranges on the table: [172, 42, 240, 99]; a black and red cord: [281, 1, 382, 213]; a round white object: [258, 156, 319, 209]; a blue plastic container: [336, 48, 383, 110]; a wooden cutting board: [39, 13, 290, 162]; a blue and silver kitchen appliance: [46, 158, 133, 213]; a silver and black coffee pot: [349, 109, 382, 213]; metal parts under desk: [194, 152, 351, 213]; the phone is black: [48, 82, 87, 131]; 
****************************************************************************************************


In [8]:
region_semantic = processor.region_semantic_model.region_semantic(image_path)

****************************************************************************************************

Step3, Semantic Prompt:
extract region segmentation with SAM model....

finished...

generate region supervision with blip/blip2 model....



╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ /home/dmcconachie/coding/Image2Paragraph/models/region_semantic.py:51 in region_semantic         │
│                                                                                                  │
│   48 │   │   print('finished...\n')                                                              │
│   49 │   │   if self.region_classify_model == 'ssa':                                             │
│   50 │   │   │   print('generate region supervision with blip/blip2 model....\n')                │
│ ❱ 51 │   │   │   anns_w_class = self.semantic_segment_model.semantic_class_w_mask(img_src, an    │
│   52 │   │   │   print('finished...\n')                                                          │
│   53 │   │   elif self.region_classify_model == 'edit_anything':                                 │
│   54 │   │   │   print('generate region supervision with edit anything model....\n')             │
│                                                                                                  │
│ /home/dmcconachie/coding/Image2Paragraph/models/segment_models/semantic_segment_anything_model.p │
│ y:120 in semantic_class_w_mask                                                                   │
│                                                                                                  │
│   117 │   │   """                                                                                │
│   118 │   │   img = mmcv.imread(img_src)                                                         │
│   119 │   │   img = resize_long_edge_cv2(img, 384)                                               │
│ ❱ 120 │   │   oneformer_coco_seg = self.oneformer_segmentation(Image.fromarray(img), self.onef   │
│   121 │   │   oneformer_ade20k_seg = self.oneformer_segmentation(Image.fromarray(img), self.on   │
│   122 │   │   bitmasks, class_names = [], []                                                     │
│   123 │   │   for ann in anns:                                                                   │
│                                                                                                  │
│ /home/dmcconachie/coding/Image2Paragraph/models/segment_models/semantic_segment_anything_model.p │
│ y:75 in oneformer_segmentation                                                                   │
│                                                                                                  │
│    72 │   │   # inputs["pixel_values"] = inputs["pixel_values"].half()                           │
│    73 │   │   # inputs["pixel_mask"] = inputs["pixel_mask"].half()                               │
│    74 │   │   # inputs["task_inputs"] = inputs["task_inputs"].half()                             │
│ ❱  75 │   │   outputs = model(**inputs)                                                          │
│    76 │   │   predicted_semantic_map = processor.post_process_semantic_segmentation(             │
│    77 │   │   │   outputs, target_sizes=[image.size[::-1]])[0]                                   │
│    78 │   │   return predicted_semantic_map                                                      │
│                                                                                                  │
│ /home/dmcconachie/miniconda3/envs/mllm_testing/lib/python3.8/site-packages/torch/nn/modules/modu │
│ le.py:1051 in _call_impl                                                                         │
│                                                                                                  │
│   1048 │   │   # this function, and just call forward.                                           │
│   1049 │   │   if not (self._backward_hooks or self._forwar